In [4]:
%matplotlib inline

In [5]:
# path = "../../homework/data/kaggle_dogs_vs_cats/"
# path = "/home/ubuntu/kaggle/dogs_vs_cats/"
path = "../../homework/data/dogscats/sample/"

In [1]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from keras.preprocessing import image as keras_image

Using Theano backend.


In [6]:
import utils; reload(utils)
from utils import plots

In [8]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [9]:
batch_size=64

In [10]:
# Prepare model and create batch
vgg = Vgg16()
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [11]:
# Finetue the model on our data
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
16/16 [==============================] - 9s - loss: 1.5024 - acc: 0.5625 - val_loss: 0.6928 - val_acc: 0.7500


In [ ]:
# test_batches = vgg.get_batches(path + 'mytest', shuffle=False, batch_size=2, class_mode=None)

In [ ]:
# img_gen = keras_image.ImageDataGenerator()
# test_batches = img_gen.flow_from_directory(path + 'mytest', target_size=(224, 224), shuffle=False, batch_size=4, class_mode=None)

In [ ]:
# imgs = next(test_batches)
# plots(imgs)

In [ ]:
# type(imgs)
# imgs.shape

In [ ]:
# vgg.predict(imgs, True)

In [ ]:
# predict class for an image file, return 0 on dog and 1 on cat
def predict_file(path):
    img_obj = keras_image.load_img(path=path, target_size=(224, 224))
    img = keras_image.img_to_array(img_obj)
    res = vgg.predict(np.array([img]), True)
    s = res[2][0]
    if s == 'cats':
        return 1
    else:
        return 0

In [ ]:
predict_file('/home/ubuntu/kaggle/dogs_vs_cats/test1/5.jpg')

In [ ]:
test_dir = path + 'test1/'
with open(path + 'result.csv', 'w') as file:
    count = 0
    for f in os.listdir(test_dir):
        count += 1
        if count % 100 == 0:
            print("count: %d" % count)
        id = f.split(".")[0]
        test_file = test_dir + f
        c = predict_file(test_file)
#         print('%s %s' % (id, c))
        file.write('%s,%s\n' % (id, c))

### The following few code blocks are copied from the course video. Run test using model's predict_generator function and save the submission file using numpy facilities.

In [22]:
test_batches = vgg.get_batches(path + 'test/', shuffle=False, batch_size=batch_size, class_mode=None)
preds = vgg.model.predict_generator(test_batches, test_batches.nb_sample)

Found 6 images belonging to 1 classes.


In [17]:
preds

array([[ 0.1623,  0.8377],
       [ 0.9666,  0.0334],
       [ 0.1286,  0.8714],
       [ 0.3864,  0.6136],
       [ 0.7654,  0.2346],
       [ 0.8961,  0.1039],
       [ 0.7703,  0.2297],
       [ 0.7038,  0.2962],
       [ 0.849 ,  0.151 ]], dtype=float32)

In [15]:
test_batches.filenames

['unknown/1.jpg',
 'unknown/d3.jpg',
 'unknown/12.jpg',
 'unknown/7.jpg',
 'unknown/d2.jpg',
 'unknown/6.jpg',
 'unknown/4.jpg',
 'unknown/5.jpg',
 'unknown/d1.jpg']

In [44]:
isdog = preds[:, 1]
isdog

array([ 0.8377,  0.8714,  0.6136,  0.1039,  0.2297,  0.2962], dtype=float32)

In [26]:
ids = [int(f[8:f.find('.')]) for f in test_batches.filenames]
ids

[1, 12, 7, 6, 4, 5]

In [40]:
subm = np.stack([ids, isdog], axis=1)
subm

array([[  1.    ,   0.8377],
       [ 12.    ,   0.8714],
       [  7.    ,   0.6136],
       [  6.    ,   0.1039],
       [  4.    ,   0.2297],
       [  5.    ,   0.2962]])

In [41]:
np.savetxt(path + 'subm.csv', subm, fmt='%d,%.5f', header='id,label')

In [42]:
from IPython.display import FileLink
FileLink(path + 'subm.csv')

/home/taot/github/fastai-courses/homework/data/dogscats/sample/subm.csv

### Clip the result so we are not so confident

In [48]:
np.clip(preds[:,1], 0.05, 0.95)
preds

array([[ 0.1623,  0.8377],
       [ 0.1286,  0.8714],
       [ 0.3864,  0.6136],
       [ 0.8961,  0.1039],
       [ 0.7703,  0.2297],
       [ 0.7038,  0.2962]], dtype=float32)

## Visialization

In [50]:
val_batches, probs = vgg.test(path + 'valid', batch_size=batch_size)

Found 8 images belonging to 2 classes.


In [51]:
labels = val_batches.classes
labels

array([0, 0, 0, 0, 1, 1, 1, 1], dtype=int32)

In [53]:
np.where(preds == labels)

/home/taot/opt/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


(array([], dtype=int64),)